> 27183 food items with reviews

** retrive Burpple data from ES **

In [1]:
%load_ext autoreload
%autoreload 2
import myfunctions as f
f.today()

'20180313'

In [4]:
# burpple parameters
burpple, burppleinitial, burpples = f.burpple_para()

In [7]:
%%time
# make sure ES is up and running
f.initialise_es(1)
print ""
website = burpples[0]
website.update({"all_rec_review" : f.retrive_from_es(website, "review", _print=True)})

{
  "name" : "star7",
  "cluster_name" : "star",
  "cluster_uuid" : "VDcV69ThThyb00sd6PgeMA",
  "version" : {
    "number" : "2.4.5",
    "build_hash" : "c849dd13904f53e63e88efc33b2ceeda0b6a1276",
    "build_timestamp" : "2017-04-24T16:18:17Z",
    "build_snapshot" : false,
    "lucene_version" : "5.5.4"
  },
  "tagline" : "You Know, for Search"
}


25709
ES location: burpple, review
Dimention: 25709 , 43
Column names::_id, _index, _score, _source.body, _source.crawlTimeStamp, _source.feedDatetime, _source.foodImgUrl, _source.id, _source.name, _source.postDatetime, _source.restaurant, _source.restaurant.address, _source.restaurant.areaUrls, _source.restaurant.avgPrice, _source.restaurant.crawlTimeStamp, _source.restaurant.id, _source.restaurant.name, _source.restaurant.openingHours, _source.restaurant.phone, _source.restaurant.tags, _source.restaurant.url, _source.restaurant.website, _source.restaurantUrl, _source.title, _source.url, _source.user, _source.user.awards, _source.user.badg

In [8]:
%%time
website = burpples[1]
website.update({"all_rec_review" : f.retrive_from_es(website, "review", _print=True)})

297480
ES location: burppleinitial, review
Dimention: 297480 , 44
Column names::_id, _index, _score, _source.areaName, _source.body, _source.crawlTimeStamp, _source.datetime, _source.foodImgUrl, _source.id, _source.name, _source.offset, _source.restaurant, _source.restaurant.address, _source.restaurant.areaUrls, _source.restaurant.avgPrice, _source.restaurant.crawlTimeStamp, _source.restaurant.id, _source.restaurant.name, _source.restaurant.openingHours, _source.restaurant.phone, _source.restaurant.tags, _source.restaurant.url, _source.restaurant.website, _source.restaurantUrl, _source.title, _source.url, _source.user, _source.user.awards, _source.user.badges, _source.user.bio, _source.user.country, _source.user.crawlTimeStamp, _source.user.name, _source.user.numOfFollowers, _source.user.numOfFollowing, _source.user.numOfReviews, _source.user.numOfWishlists, _source.user.url, _source.user.userImgUrl, _source.user.username, _source.user.website, _source.userImgUrl, _source.username, _ty

** consolidate burpple records **

In [10]:
%%time
df = f.consolidate_burpple_records(burpples)

Got 323104 unique records of review
Got 323051 unique records of images
Got 11180 unique users
Got 14547 unique vendors
Review time from 2011-11-08 00:00:00.000000 to 2017-07-20 23:04:45.837391
CPU times: user 8.06 s, sys: 276 ms, total: 8.33 s
Wall time: 8.33 s


In [11]:
df.head(3)

,_id,_type,_index,_source.title,_source.body,_source.crawlTimeStamp,_source.foodImgUrl,_source.username,_source.restaurant.id,_source.restaurant.name,feedTime
0,--0EG2cp,review,burppleinitial,Sicilian Lemon Creme andCaramelized Red Mixed ...,,2017-04-03 11:00:52.282519,https://s3.burpple.com/foods/376d87a944deb0499...,@ieatkows,34166,OSO Ristorante,2014-12-22 00:00:00.000000
1,--0ck5YP,review,burppleinitial,Just look at their soy sauce chicken.,The tender and juicy thigh meat with their fra...,2017-04-03 11:00:52.282519,https://s3.burpple.com/foods/2fbab5e890cf97f31...,@jiaknonstop,164762,Fatty Ox Hong Kong Kitchen (Chinatown Complex ...,2014-12-15 00:00:00.000000
2,--0gMGRO,review,burppleinitial,"Holy, smokin' Burnt Corn Tacos - it's their WE...",(Read: time for MORE alcohol.) #AntiDoteBar #...,2017-04-03 11:00:52.282519,https://s3.burpple.com/foods/16ff778f20e5f8614...,@Jazpster,539,Asian Market Caf (Fairmont Singapore),2016-11-18 00:00:00.000000


In [13]:
%%time
file_name = "burpple__review__unique__"+f.today()+".p"
f.save_file(df, file_name)

burpple__review__unique__20180313.p
CPU times: user 17.5 s, sys: 452 ms, total: 18 s
Wall time: 18 s


** obtain review objects with clean text **

In [4]:
# add new columns
import pandas as pd
pd.options.mode.chained_assignment = None # default is warn
# new col: loc
df["loc"] = df["_index"] + "/" + df["_type"] + "/" + df["_id"] 
cols = ['loc','_source.title','_source.body','_source.restaurant.name']
df = df[cols]
rename_cols = [col.replace("_source.", "") for col in cols]
df.columns = rename_cols
# new col: full_review
df["full_review"] = df['title'] + " " + df['body']

In [6]:
%%time
# twitter style cleaning
df["review_text"], df["hashtags"], df["mentions"], df["dollars"], df["scores"], df["urls"] = \
zip(*df["full_review"].apply(f.clean_review))
cols = df.columns.tolist()
cols.remove("full_review")
df = df[cols]

CPU times: user 15.3 s, sys: 652 ms, total: 16 s
Wall time: 15.8 s


In [7]:
%%time
# On review text, perform clean_name_v2()
df["clean_review"] = df["review_text"].apply(f.clean_name_v2)

CPU times: user 22.8 s, sys: 56 ms, total: 22.9 s
Wall time: 22.9 s


In [8]:
print df.shape
print df.columns.tolist()
df.sample(2,random_state=2008)

(323104, 11)
['loc', 'title', 'body', 'restaurant.name', 'review_text', 'hashtags', 'mentions', 'dollars', 'scores', 'urls', 'clean_review']


,loc,title,body,restaurant.name,review_text,hashtags,mentions,dollars,scores,urls,clean_review
242662,burppleinitial/review/jnF_zvaY,"Aged Cheddar, Fig & #Cognac Jam and Candied Wa...","Aged Cheddar, Fig & #Cognac Jam and Candied Wa...",The Humble Loaf,"Aged Cheddar, Fig and Jam and Candied Walnuts ...","(#Cognac, #lunch, #weekend, #Cognac, #lunch, #...",(),(),(),(),aged cheddar fig and jam and candied walnuts o...
74846,burppleinitial/review/CzDetUln,Orange Brioche,,Artisan Boulangerie Co. (Great World City),Orange Brioche,(),(),(),(),(),orange brioche


In [9]:
%%time
file_name = "burpple__review__clean__"+f.today()+".p"
f.save_file(df, file_name)

saved: burpple__review__clean__20180317.p
CPU times: user 17.3 s, sys: 632 ms, total: 17.9 s
Wall time: 17.9 s


In [10]:
all_reviews = sorted([s for s in set(df["clean_review"].tolist()) if len(s)>0])
print "number of unique reviews: %d"%len(all_reviews)

number of unique reviews: 277772


** retrive food entities **

In [13]:
file_name = "food_entities__20180317.p"
food = f.retrive_file(file_name)

retrived: food_entities__20180317.p


In [22]:
food_entities = food.reset_index()
# remove food of " and "-PRON-"
food_entities = food_entities[~food_entities["assigned_name"].isin(["", "-PRON-"])]
# food name variant : food_item_index
index_ref = dict()
def add_ref(line):
    v = line["index"]
    index_ref.update({k:v for k in line["name_variants"]})
food_entities.apply(add_ref, axis =1)
food_items = index_ref.keys()
print "number of food searched for: %d"%len(food_items)

number of food searched for: 139083


In [23]:
file_name = "words_to_remove_20180308.csv"
rmv = pd.read_csv(file_name, header=None)[0].tolist()
rmv = rmv+['can', 'sweet', 'love', 'up', 'no', 'me', 'back', 'fluffy', 'hot', 'spicy', 'crisp', 'salty', 'house', 'red', 'fat', 'roasted', 'white', 'refreshing', 'black', 'cold', 'deep fried', 'pieces', 'toasted', 'skin', 'cheesy', 'hungry', 'pop', 'miss', 'option', 'fried', 'tomato', 'vanilla', 'plus', 'potato', 'thai', 'spice', 'infused', 'sauces', 'to share', 'lover', 'surprise', 'balanced', 'choice', 'french', 'green', 'steamed', 'italian', 'wake up', 'silver', 'savoury flavours', 'western', 'set for', 'pan fried', 'the original', 'decadence', 'the great', 'uplifting', 'all in', 'meaty', 'spice level', 'per slice', 'discovery', 'smokey', 'all time favourites', 'sticky', 'the double', 'pint', 'mad', 'singapore style', 'alcoholic', 'serves', 'nutty', 'salted', 'lunch set', 'boost', 'anything', 'dips', 'vietnamese', 'set lunch', 'lager', 'for pax', 'below', 'bundle']

** search for food entities **

In [30]:
%%time
"""
saved: burpple__review__food__20180318.p
CPU times: user 4h 48min 43s, sys: 1min 31s, total: 4h 50min 15s
Wall time: 4h 47min 42s
"""
def search_food(review):
    return [s for s in f.search_food(review, values=food_items) if s not in rmv]

if 1==1:
    df["food_items"] = df["clean_review"].apply(search_food)    
    # ref name_variation to food index
    df["food_index"] = df["food_items"].apply(lambda l: 
                                              sorted([index_ref[s] for s in l]))
    # remove original review_text
    cols = df.columns.tolist()
    cols.remove("review_text")
    df = df[cols]

saved: burpple__review__food__20180318.p
CPU times: user 4h 48min 43s, sys: 1min 31s, total: 4h 50min 15s
Wall time: 4h 47min 42s


In [59]:
file_name = "burpple__review__food__"+f.today()+".p" 
f.save_file(df, file_name)

saved: burpple__review__food__20180319.p


In [76]:
df.columns.tolist()

['loc',
 'title',
 'body',
 'restaurant.name',
 'hashtags',
 'mentions',
 'dollars',
 'scores',
 'urls',
 'clean_review',
 'food_items',
 'food_index']

In [58]:
df.sample(2, random_state=2008)

,loc,title,body,restaurant.name,hashtags,mentions,dollars,scores,urls,clean_review,food_items,food_index
242662,burppleinitial/review/jnF_zvaY,"Aged Cheddar, Fig & #Cognac Jam and Candied Wa...","Aged Cheddar, Fig & #Cognac Jam and Candied Wa...",The Humble Loaf,"(#Cognac, #lunch, #weekend, #Cognac, #lunch, #...",(),(),(),(),aged cheddar fig and jam and candied walnuts o...,"[toast, jam, walnuts]","[food_item_102154, food_item_47881, food_item_..."
74846,burppleinitial/review/CzDetUln,Orange Brioche,,Artisan Boulangerie Co. (Great World City),(),(),(),(),(),orange brioche,[orange brioche],[food_item_65293]


In [61]:
freq = f.count_freq(f.flatten(df["food_items"].tolist()))
print "number of food name variations itendtified: %d"%len(freq)

number of food name variations itendtified: 27183


In [60]:
freq = f.count_freq(f.flatten(df["food_index"].tolist()))
print "number of unique food items itendtified: %d"%len(freq)

number of unique food items itendtified: 21273


In [41]:
# most popular food items
freq[:10]

[('coffee', 9807),
 ('meat', 7097),
 ('ice cream', 5983),
 ('rice', 5537),
 ('soup', 5145),
 ('egg', 4536),
 ('noodles', 4500),
 ('cake', 4016),
 ('cheese', 3925),
 ('pork', 3771)]

** link review locs to entities ** 

In [42]:
# obtain review locs for each food item
loc_ref = df.set_index("loc").to_dict()["food_index"]
inv_loc_ref = f.inverse_dict(loc_ref)
# append review locs to food entities
food.reset_index(inplace=True)
def map_loc(s):
    try:
        return inv_loc_ref[s]
    except:
        return []
food["review_loc"] = food["index"].apply(map_loc)
food.set_index("index", inplace=True)

In [73]:
food.sample(2, random_state=2008)

,assigned_name,name_variants,menu_name,tag,price,desc,vendors,identification_index,loc,price_average,review_loc,first_record,latest_record
index,,,,,,,,,,,,,
food_item_13994,cannelloni di malanzane,[cannelloni di malanzane],"[(Cannelloni Di Malanzane, 15)]",[Specialita],"[31.03, 31.03, 31.03, 31.03, 31.03, 31.03, 31....","[, Baked eggplant cannelloni, goat ricotta che...","[delivery_3055, delivery_3056]","(food_17362,)","[deliveroo/food/39541, deliveroo/food/82634, d...",31.03,[],2017-07-03,2018-03-01
food_item_6491,beef mandi,[beef mandi],"[(Beef Mandi, 18)]","[Favourites, Mandi]","[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","[, Default]",[delivery_199],"(food_8022,)","[foodpanda/menu_item/112620, foodpanda/menu_it...",8.00,[],2017-07-11,2018-03-01


In [74]:
## save to json
file_name = "__".join(["food_entities", f.today()]) + ".json"
food.to_json(file_name, orient='index')
print "saved: %s"%file_name

saved: food_entities__20180320.json


In [75]:
## save to p file
file_name = "__".join(["food_entities", f.today()]) + ".p"
f.save_file(food, file_name)

saved: food_entities__20180320.p
